# 🧩 Phase 3: GEDIG Maze - Large Scale GPU Experiment

このノートブックでは、GPU最適化されたColab環境で、GEDIG（Graph Edit Distance + Information Gain）迷路解法アルゴリズムの大規模実験を行います。

## 🎯 実験目標
- **試行回数削減**: 60%削減
- **収束速度**: 3倍高速収束  
- **成功率**: 95%達成

複数のアルゴリズム（A*, Dijkstra, Q-Learning, 遺伝的アルゴリズム、SlimeMold_GEDIG）を比較し、GEDIG手法の優位性を実証します。

## 🔧 統一セットアップ
InsightSpike-AI統一セットアップスクリプトでGEDIG迷路実験環境を準備します。

In [ ]:
# Phase3用統一セットアップ実行
import os
import subprocess
import sys

# InsightSpike-AIリポジトリの準備
if not os.path.exists('/content/InsightSpike-AI'):
    print("📂 InsightSpike-AIリポジトリをクローン中...")
    !git clone https://github.com/miyauchi-kazuyoshi/InsightSpike-AI.git /content/InsightSpike-AI
    os.chdir('/content/InsightSpike-AI')
else:
    print("📂 リポジトリは既に存在します")
    os.chdir('/content/InsightSpike-AI')

print("🚀 Phase3用統一セットアップ実行中...")

try:
    # 統一セットアップスクリプト実行
    !chmod +x scripts/colab/setup_unified.sh
    result = subprocess.run(['bash', 'scripts/colab/setup_unified.sh'], 
                          capture_output=True, text=True, cwd='/content/InsightSpike-AI')
    
    print("📋 統一セットアップ出力:")
    print(result.stdout)
    
    if result.stderr:
        print("⚠️ 警告:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✅ 統一セットアップ完了!")
    else:
        print("❌ セットアップでエラーが発生しました")

except Exception as e:
    print(f"⚠️ 統一セットアップ失敗: {e}")

# Phase3専用ライブラリ追加インストール
print("📦 Phase3専用GEDIG迷路ライブラリを追加インストール中...")
phase3_specific_libraries = [
    "networkx",
    "scipy",
    "scikit-learn", 
    "imageio",
    "pillow",
    "plotly",
    "matplotlib",
    "seaborn",
    "torch",
    "numpy",
    "tqdm"
]

for lib in phase3_specific_libraries:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "--upgrade", "-q"])
        print(f"✅ {lib}")
    except Exception as e:
        print(f"⚠️ {lib} インストール失敗: {e}")

# Python パス設定
sys.path.insert(0, '/content/InsightSpike-AI/src')
sys.path.insert(0, '/content/InsightSpike-AI/experiments_colab/shared')

# GPU環境確認
import torch
import numpy as np
import random

print(f"\n🎮 GPU環境:")
print(f"  CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU名: {torch.cuda.get_device_name(0)}")
    print(f"  GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# ランダムシード設定（再現性のため）
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("🎲 ランダムシード設定完了")

print("🎯 Phase3 GEDIG迷路実験環境セットアップ完了!")

## 🔧 環境セットアップ
GPU最適化されたColab環境でGEDIG迷路実験の準備を行います。

In [ ]:
# GPU環境確認とライブラリインストール
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import sys
import time
import random
from collections import deque, defaultdict
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any, Optional
import heapq
import copy

print("🔍 Colab環境チェック")
print("=" * 50)

# GPU確認
if torch.cuda.is_available():
    print(f"✅ GPU利用可能: {torch.cuda.get_device_name(0)}")
    print(f"📊 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
else:
    print("❌ GPU利用不可 - CPUで実行します")
    device = torch.device('cpu')

print(f"🔧 PyTorch Version: {torch.__version__}")
print(f"🎯 使用デバイス: {device}")

# 必要なライブラリのインストール
libraries = [
    "networkx",
    "scipy", 
    "scikit-learn",
    "imageio",
    "pillow",
    "plotly"
]

print("\n📦 ライブラリインストール中...")
for lib in libraries:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", lib, "-q"])
        print(f"✅ {lib}")
    except Exception as e:
        print(f"⚠️ {lib} インストール失敗: {e}")

# 追加のインポート
import networkx as nx
from scipy.spatial.distance import euclidean
from sklearn.metrics import adjusted_rand_score

print("\n🚀 環境セットアップ完了!")

# ランダムシード設定（再現性のため）
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
print("🎲 ランダムシード設定完了")

## 🗺️ 迷路環境とアルゴリズム実装
多様な迷路環境と5つの解法アルゴリズムを実装します。

In [ ]:
# 迷路環境クラスと基本アルゴリズム実装

@dataclass
class MazeResult:
    algorithm: str
    maze_type: str
    maze_size: Tuple[int, int]
    success: bool
    steps: int
    path_length: int
    execution_time: float
    path: List[Tuple[int, int]]
    solution_quality: float = 0.0

class Maze:
    """迷路環境クラス"""
    
    def __init__(self, width: int, height: int, maze_type: str = "simple"):
        self.width = width
        self.height = height
        self.maze_type = maze_type
        self.grid = np.ones((height, width), dtype=int)  # 1=通路, 0=壁
        self.start = (0, 0)
        self.goal = (height-1, width-1)
        self.dynamic_obstacles = []
        self.multiple_goals = []
        
        self._generate_maze()
    
    def _generate_maze(self):
        """迷路生成"""
        if self.maze_type == "simple":
            self._generate_simple_maze()
        elif self.maze_type == "complex":
            self._generate_complex_maze()
        elif self.maze_type == "dynamic":
            self._generate_dynamic_maze()
        elif self.maze_type == "multigoal":
            self._generate_multigoal_maze()
    
    def _generate_simple_maze(self):
        """シンプル迷路生成"""
        # ランダムに壁を配置
        wall_density = 0.3
        for i in range(self.height):
            for j in range(self.width):
                if random.random() < wall_density:
                    self.grid[i, j] = 0
        
        # スタートとゴールは通路に
        self.grid[self.start] = 1
        self.grid[self.goal] = 1
        
        # 経路が存在することを保証
        self._ensure_path_exists()
    
    def _generate_complex_maze(self):
        """複雑迷路生成"""
        # より複雑なパターン
        wall_density = 0.4
        
        # パターン1: 迷路状の構造
        for i in range(0, self.height, 2):
            for j in range(self.width):
                self.grid[i, j] = 0
        
        for i in range(self.height):
            for j in range(0, self.width, 3):
                self.grid[i, j] = 0
        
        # ランダムな開口部
        for _ in range(self.width * self.height // 10):
            i, j = random.randint(0, self.height-1), random.randint(0, self.width-1)
            self.grid[i, j] = 1
        
        self.grid[self.start] = 1
        self.grid[self.goal] = 1
        self._ensure_path_exists()
    
    def _generate_dynamic_maze(self):
        """動的障害物迷路"""
        self._generate_simple_maze()
        
        # 動的障害物の追加
        num_dynamic = min(5, self.width * self.height // 20)
        for _ in range(num_dynamic):
            pos = (random.randint(0, self.height-1), random.randint(0, self.width-1))
            if pos != self.start and pos != self.goal:
                self.dynamic_obstacles.append(pos)
    
    def _generate_multigoal_maze(self):
        """複数ゴール迷路"""
        self._generate_simple_maze()
        
        # 複数ゴールの設定
        num_goals = min(3, max(2, self.width * self.height // 30))
        for _ in range(num_goals):
            goal_pos = (random.randint(0, self.height-1), random.randint(0, self.width-1))
            if goal_pos != self.start and self.grid[goal_pos] == 1:
                self.multiple_goals.append(goal_pos)
        
        # 元のゴールも含める
        if self.goal not in self.multiple_goals:
            self.multiple_goals.append(self.goal)
    
    def _ensure_path_exists(self):
        """経路存在を保証（DFSで接続性確認）"""
        visited = set()
        stack = [self.start]
        
        while stack:
            pos = stack.pop()
            if pos in visited:
                continue
            visited.add(pos)
            
            if pos == self.goal:
                return  # 経路存在
            
            for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
                ni, nj = pos[0] + di, pos[1] + dj
                if (0 <= ni < self.height and 0 <= nj < self.width and 
                    self.grid[ni, nj] == 1 and (ni, nj) not in visited):
                    stack.append((ni, nj))
        
        # 経路が存在しない場合、強制的に作成
        self._create_path()
    
    def _create_path(self):
        """強制的に経路作成"""
        # シンプルな直線経路を作成
        i, j = self.start
        while (i, j) != self.goal:
            self.grid[i, j] = 1
            if i < self.goal[0]:
                i += 1
            elif j < self.goal[1]:
                j += 1
    
    def is_valid_position(self, pos: Tuple[int, int], step: int = 0) -> bool:
        """位置の有効性チェック"""
        i, j = pos
        if not (0 <= i < self.height and 0 <= j < self.width):
            return False
        if self.grid[i, j] == 0:
            return False
        
        # 動的障害物チェック
        if self.maze_type == "dynamic":
            for obs_pos in self.dynamic_obstacles:
                # 時間によって動く障害物（簡単な例）
                moving_pos = ((obs_pos[0] + step // 5) % self.height, obs_pos[1])
                if pos == moving_pos:
                    return False
        
        return True
    
    def get_neighbors(self, pos: Tuple[int, int], step: int = 0) -> List[Tuple[int, int]]:
        """隣接する有効な位置を取得"""
        neighbors = []
        for di, dj in [(0,1), (0,-1), (1,0), (-1,0)]:
            new_pos = (pos[0] + di, pos[1] + dj)
            if self.is_valid_position(new_pos, step):
                neighbors.append(new_pos)
        return neighbors
    
    def is_goal(self, pos: Tuple[int, int]) -> bool:
        """ゴール判定"""
        if self.maze_type == "multigoal":
            return pos in self.multiple_goals
        return pos == self.goal
    
    def visualize(self, path: List[Tuple[int, int]] = None, title: str = ""):
        """迷路可視化"""
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # 迷路描画
        maze_visual = self.grid.copy().astype(float)
        
        # 壁を黒、通路を白に
        maze_visual = 1 - maze_visual  # 反転
        
        # スタートを緑、ゴールを赤に
        maze_visual = np.stack([maze_visual, maze_visual, maze_visual], axis=2)
        maze_visual[self.start] = [0, 1, 0]  # 緑
        
        if self.maze_type == "multigoal":
            for goal in self.multiple_goals:
                maze_visual[goal] = [1, 0, 0]  # 赤
        else:
            maze_visual[self.goal] = [1, 0, 0]  # 赤
        
        # 経路を青で描画
        if path:
            for pos in path[1:-1]:  # スタートとゴール以外
                maze_visual[pos] = [0, 0, 1]  # 青
        
        # 動的障害物を紫で描画
        if self.maze_type == "dynamic":
            for obs_pos in self.dynamic_obstacles:
                maze_visual[obs_pos] = [0.5, 0, 0.5]  # 紫
        
        ax.imshow(maze_visual)
        ax.set_title(f"{title}\n{self.maze_type} maze ({self.width}x{self.height})")
        ax.set_xticks([])
        ax.set_yticks([])
        
        return fig

print("✅ 迷路環境クラス実装完了")

# InsightSpike-AI モジュールのインポート確認
try:
    # 正しいモジュールパスでインポート
    from insightspike.core.agents.main_agent import MainAgent
    from insightspike.core.learning.knowledge_graph_memory import KnowledgeGraphMemory
    from insightspike.config import Config
    from insightspike.utils import logger
    
    print("✅ InsightSpike-AI modules imported successfully!")
    print("Available modules:")
    print("- MainAgent:", MainAgent)
    print("- KnowledgeGraphMemory:", KnowledgeGraphMemory)
    print("- Config:", Config)
    print("- logger:", logger)
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Available insightspike modules:")
    import insightspike
    print(dir(insightspike))
    
    # モジュール構造を確認
    import os
    import sys
    for path in sys.path:
        if 'InsightSpike-AI' in path:
            print(f"Python path: {path}")
            if os.path.exists(path + '/insightspike'):
                print("  - insightspike found")
                print("  - submodules:", os.listdir(path + '/insightspike'))

In [ ]:
# 🧩 Phase 3: GEDIG Maze - Large Scale GPU Experiment

このノートブックでは、GPU最適化されたColab環境で、GEDIG（Graph Edit Distance + Information Gain）迷路解法アルゴリズムの大規模実験を行います。

## 🎯 実験目標
- **試行回数削減**: 60%削減
- **収束速度**: 3倍高速収束  
- **成功率**: 95%達成

複数のアルゴリズム（A*, Dijkstra, Q-Learning, 遺伝的アルゴリズム、SlimeMold_GEDIG）を比較し、GEDIG手法の優位性を実証します。

# GED-IG迷路ナビゲーション実験の実装
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from typing import List, Tuple, Dict, Any
import random
import time
import json
from datetime import datetime

class MazeEnvironment:
    """迷路環境クラス"""
    
    def __init__(self, size: Tuple[int, int] = (10, 10), wall_probability: float = 0.3):
        self.size = size
        self.maze = self._generate_maze(wall_probability)
        self.start = (0, 0)
        self.goal = (size[0]-1, size[1]-1)
        self.current_pos = self.start
        
    def _generate_maze(self, wall_probability: float) -> np.ndarray:
        """迷路を生成（0: 通路, 1: 壁）"""
        maze = np.random.random(self.size) < wall_probability
        maze = maze.astype(int)
        maze[0, 0] = 0  # スタート地点
        maze[-1, -1] = 0  # ゴール地点
        return maze
    
    def get_neighbors(self, pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        """隣接する通路セルを取得"""
        x, y = pos
        neighbors = []
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x + dx, y + dy
            if (0 <= nx < self.size[0] and 0 <= ny < self.size[1] and 
                self.maze[nx, ny] == 0):
                neighbors.append((nx, ny))
        return neighbors
    
    def reset(self):
        """環境をリセット"""
        self.current_pos = self.start
        return self.current_pos
    
    def step(self, action: Tuple[int, int]) -> Tuple[Tuple[int, int], float, bool]:
        """アクションを実行"""
        if action in self.get_neighbors(self.current_pos):
            self.current_pos = action
            reward = 10 if self.current_pos == self.goal else -0.1
            done = self.current_pos == self.goal
            return self.current_pos, reward, done
        else:
            return self.current_pos, -1, False  # 無効な移動

class GEDIGNavigator:
    """GED-IG（Graph Edit Distance - Information Gain）ナビゲーター"""
    
    def __init__(self, maze_env: MazeEnvironment):
        self.env = maze_env
        self.visited_states = set()
        self.graph = nx.Graph()
        self.path_history = []
        
    def calculate_information_gain(self, state: Tuple[int, int]) -> float:
        """情報ゲインを計算"""
        if state in self.visited_states:
            return 0.0
        
        # 未訪問状態の情報価値
        neighbors = self.env.get_neighbors(state)
        unexplored_neighbors = len([n for n in neighbors if n not in self.visited_states])
        
        # ゴールまでの距離を考慮
        goal_distance = abs(state[0] - self.env.goal[0]) + abs(state[1] - self.env.goal[1])
        
        # 情報ゲイン = 探索価値 - 距離ペナルティ
        info_gain = unexplored_neighbors * 2.0 - goal_distance * 0.1
        return max(info_gain, 0.1)
    
    def calculate_graph_edit_distance(self, current_graph: nx.Graph, 
                                    new_state: Tuple[int, int]) -> float:
        """グラフ編集距離を計算"""
        # 新しい状態を追加した場合のグラフ構造変化
        temp_graph = current_graph.copy()
        temp_graph.add_node(new_state)
        
        # 現在位置との接続を追加
        for neighbor in self.env.get_neighbors(new_state):
            if neighbor in temp_graph.nodes:
                temp_graph.add_edge(new_state, neighbor)
        
        # 構造的複雑さを計算
        clustering = nx.average_clustering(temp_graph) if len(temp_graph) > 2 else 0
        connectivity = len(temp_graph.edges) / max(len(temp_graph.nodes), 1)
        
        return clustering + connectivity
    
    def select_next_action(self, current_state: Tuple[int, int]) -> Tuple[int, int]:
        """次のアクションを選択"""
        neighbors = self.env.get_neighbors(current_state)
        if not neighbors:
            return current_state
        
        # 各隣接状態のスコアを計算
        scores = {}
        for neighbor in neighbors:
            info_gain = self.calculate_information_gain(neighbor)
            ged = self.calculate_graph_edit_distance(self.graph, neighbor)
            
            # スコア = 情報ゲイン + グラフ構造最適化
            scores[neighbor] = info_gain + ged
        
        # 最高スコアの状態を選択
        best_action = max(scores.keys(), key=lambda x: scores[x])
        return best_action
    
    def navigate(self, max_steps: int = 100) -> Dict[str, Any]:
        """迷路をナビゲート"""
        current_state = self.env.reset()
        self.visited_states = {current_state}
        self.graph.add_node(current_state)
        self.path_history = [current_state]
        
        total_reward = 0
        steps = 0
        
        for step in range(max_steps):
            if current_state == self.env.goal:
                break
                
            next_action = self.select_next_action(current_state)
            next_state, reward, done = self.env.step(next_action)
            
            # グラフを更新
            if next_state not in self.graph.nodes:
                self.graph.add_node(next_state)
            self.graph.add_edge(current_state, next_state)
            
            self.visited_states.add(next_state)
            self.path_history.append(next_state)
            current_state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        return {
            'success': current_state == self.env.goal,
            'steps': steps,
            'total_reward': total_reward,
            'path_length': len(self.path_history),
            'exploration_ratio': len(self.visited_states) / (self.env.size[0] * self.env.size[1]),
            'path': self.path_history
        }

# 実験実行
print("🚀 GED-IG迷路ナビゲーション実験開始")

# 実験設定
maze_sizes = [(5, 5), (10, 10), (15, 15)]
wall_probabilities = [0.2, 0.3, 0.4]
num_trials = 5

results = []

for size in maze_sizes:
    for wall_prob in wall_probabilities:
        print(f"\n📊 実験設定: {size} maze, {wall_prob:.1f} wall probability")
        
        trial_results = []
        for trial in range(num_trials):
            # 迷路環境を作成
            maze_env = MazeEnvironment(size, wall_prob)
            navigator = GEDIGNavigator(maze_env)
            
            # ナビゲーション実行
            result = navigator.navigate()
            trial_results.append(result)
            
            print(f"  Trial {trial+1}: {'SUCCESS' if result['success'] else 'FAILED'} "
                  f"({result['steps']} steps, {result['total_reward']:.2f} reward)")
        
        # 統計計算
        success_rate = sum(1 for r in trial_results if r['success']) / num_trials
        avg_steps = np.mean([r['steps'] for r in trial_results])
        avg_reward = np.mean([r['total_reward'] for r in trial_results])
        avg_exploration = np.mean([r['exploration_ratio'] for r in trial_results])
        
        results.append({
            'maze_size': size,
            'wall_probability': wall_prob,
            'success_rate': success_rate,
            'avg_steps': avg_steps,
            'avg_reward': avg_reward,
            'avg_exploration_ratio': avg_exploration,
            'trials': trial_results
        })

print("\n✅ GED-IG迷路ナビゲーション実験完了")

## 📊 実験結果の可視化

In [ ]:
# 実験結果の可視化
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# スタイル設定
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 1. 成功率の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🎯 GED-IG迷路ナビゲーション実験結果', fontsize=16, fontweight='bold')

# 成功率ヒートマップ
success_data = np.array([[r['success_rate'] for r in results if r['maze_size'] == size] 
                        for size in maze_sizes])

ax1 = axes[0, 0]
im1 = ax1.imshow(success_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax1.set_title('成功率 (Success Rate)')
ax1.set_xlabel('Wall Probability')
ax1.set_ylabel('Maze Size')
ax1.set_xticks(range(len(wall_probabilities)))
ax1.set_xticklabels([f'{p:.1f}' for p in wall_probabilities])
ax1.set_yticks(range(len(maze_sizes)))
ax1.set_yticklabels([f'{s[0]}x{s[1]}' for s in maze_sizes])

# 成功率の値をテキストで表示
for i in range(len(maze_sizes)):
    for j in range(len(wall_probabilities)):
        text = ax1.text(j, i, f'{success_data[i, j]:.2f}',
                       ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im1, ax=ax1)

# 平均ステップ数
avg_steps_data = np.array([[r['avg_steps'] for r in results if r['maze_size'] == size] 
                          for size in maze_sizes])

ax2 = axes[0, 1]
im2 = ax2.imshow(avg_steps_data, cmap='viridis', aspect='auto')
ax2.set_title('平均ステップ数 (Average Steps)')
ax2.set_xlabel('Wall Probability')
ax2.set_ylabel('Maze Size')
ax2.set_xticks(range(len(wall_probabilities)))
ax2.set_xticklabels([f'{p:.1f}' for p in wall_probabilities])
ax2.set_yticks(range(len(maze_sizes)))
ax2.set_yticklabels([f'{s[0]}x{s[1]}' for s in maze_sizes])

for i in range(len(maze_sizes)):
    for j in range(len(wall_probabilities)):
        text = ax2.text(j, i, f'{avg_steps_data[i, j]:.1f}',
                       ha="center", va="center", color="white", fontweight='bold')

plt.colorbar(im2, ax=ax2)

# 探索効率の比較
ax3 = axes[1, 0]
exploration_data = [r['avg_exploration_ratio'] for r in results]
maze_labels = [f"{r['maze_size'][0]}x{r['maze_size'][1]}, {r['wall_probability']:.1f}" 
               for r in results]

bars = ax3.bar(range(len(exploration_data)), exploration_data, 
               color=plt.cm.Set3(np.linspace(0, 1, len(exploration_data))))
ax3.set_title('探索効率 (Exploration Ratio)')
ax3.set_xlabel('実験設定')
ax3.set_ylabel('探索率')
ax3.set_xticks(range(len(exploration_data)))
ax3.set_xticklabels(maze_labels, rotation=45, ha='right')

# 値をバーの上に表示
for i, (bar, value) in enumerate(zip(bars, exploration_data)):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# 迷路の例を可視化
ax4 = axes[1, 1]
# 最後の実験の迷路とパスを可視化
last_result = results[-1]['trials'][0]
if 'path' in last_result:
    # 迷路環境を再作成
    example_maze = MazeEnvironment(maze_sizes[-1], wall_probabilities[-1])
    
    # 迷路を描画
    ax4.imshow(example_maze.maze, cmap='binary', alpha=0.7)
    
    # パスを描画
    if last_result['path']:
        path_x = [p[1] for p in last_result['path']]
        path_y = [p[0] for p in last_result['path']]
        ax4.plot(path_x, path_y, 'ro-', linewidth=3, markersize=6, alpha=0.8, label='Path')
    
    # スタートとゴールをマーク
    ax4.plot(example_maze.start[1], example_maze.start[0], 'go', markersize=12, label='Start')
    ax4.plot(example_maze.goal[1], example_maze.goal[0], 'bs', markersize=12, label='Goal')
    
    ax4.set_title('迷路例とナビゲーションパス')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. 詳細統計の表示
print("\n📈 詳細統計情報:")
print("=" * 80)

for i, result in enumerate(results):
    size = result['maze_size']
    wall_prob = result['wall_probability']
    
    print(f"\n🎯 実験 {i+1}: {size[0]}x{size[1]} 迷路, 壁確率 {wall_prob:.1f}")
    print(f"   成功率: {result['success_rate']:.2%}")
    print(f"   平均ステップ数: {result['avg_steps']:.1f}")
    print(f"   平均報酬: {result['avg_reward']:.2f}")
    print(f"   平均探索率: {result['avg_exploration_ratio']:.3f}")
    
    # 個別試行の詳細
    for j, trial in enumerate(result['trials']):
        status = "✅" if trial['success'] else "❌"
        print(f"     試行 {j+1}: {status} {trial['steps']} steps, "
              f"{trial['total_reward']:.2f} reward, "
              f"{trial['exploration_ratio']:.3f} exploration")

print("\n🎉 可視化完了!")

## 💾 実験結果の保存

In [ ]:
# 実験結果の保存
import os
import json
from datetime import datetime

# 保存ディレクトリの作成
output_dir = "/content/phase3_results"
os.makedirs(output_dir, exist_ok=True)

# タイムスタンプ付きファイル名
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = os.path.join(output_dir, f"gedig_maze_results_{timestamp}.json")
summary_file = os.path.join(output_dir, f"gedig_maze_summary_{timestamp}.txt")

# JSON形式で詳細結果を保存
results_to_save = {
    'experiment_info': {
        'name': 'GED-IG Maze Navigation',
        'description': 'Graph Edit Distance - Information Gain based maze navigation experiment',
        'timestamp': timestamp,
        'colab_environment': True,
        'gpu_accelerated': torch.cuda.is_available()
    },
    'experiment_settings': {
        'maze_sizes': maze_sizes,
        'wall_probabilities': wall_probabilities,
        'num_trials': num_trials
    },
    'results': results,
    'overall_statistics': {
        'total_experiments': len(results),
        'overall_success_rate': np.mean([r['success_rate'] for r in results]),
        'avg_steps_across_all': np.mean([r['avg_steps'] for r in results]),
        'avg_exploration_across_all': np.mean([r['avg_exploration_ratio'] for r in results])
    }
}

with open(results_file, 'w') as f:
    json.dump(results_to_save, f, indent=2, default=str)

print(f"✅ 詳細結果を保存: {results_file}")

# テキスト形式でサマリーを保存
with open(summary_file, 'w') as f:
    f.write("GED-IG Maze Navigation Experiment Summary\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"実験日時: {timestamp}\n")
    f.write(f"Google Colab環境: {True}\n")
    f.write(f"GPU利用可能: {torch.cuda.is_available()}\n\n")
    
    f.write("実験設定:\n")
    f.write(f"- 迷路サイズ: {maze_sizes}\n")
    f.write(f"- 壁確率: {wall_probabilities}\n")
    f.write(f"- 試行回数: {num_trials}\n\n")
    
    f.write("全体統計:\n")
    f.write(f"- 総実験数: {len(results)}\n")
    f.write(f"- 全体成功率: {np.mean([r['success_rate'] for r in results]):.2%}\n")
    f.write(f"- 平均ステップ数: {np.mean([r['avg_steps'] for r in results]):.1f}\n")
    f.write(f"- 平均探索率: {np.mean([r['avg_exploration_ratio'] for r in results]):.3f}\n\n")
    
    f.write("詳細結果:\n")
    for i, result in enumerate(results):
        size = result['maze_size']
        wall_prob = result['wall_probability']
        f.write(f"\n実験 {i+1}: {size[0]}x{size[1]} 迷路, 壁確率 {wall_prob:.1f}\n")
        f.write(f"  成功率: {result['success_rate']:.2%}\n")
        f.write(f"  平均ステップ数: {result['avg_steps']:.1f}\n")
        f.write(f"  平均報酬: {result['avg_reward']:.2f}\n")
        f.write(f"  平均探索率: {result['avg_exploration_ratio']:.3f}\n")

print(f"✅ サマリーを保存: {summary_file}")

# Google Driveへのバックアップ（オプション）
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Drive上の保存先
    drive_dir = "/content/drive/MyDrive/InsightSpike_Experiments/Phase3"
    os.makedirs(drive_dir, exist_ok=True)
    
    # ファイルをコピー
    import shutil
    shutil.copy(results_file, drive_dir)
    shutil.copy(summary_file, drive_dir)
    
    print(f"✅ Google Driveにバックアップ: {drive_dir}")
except Exception as e:
    print(f"⚠️ Google Driveバックアップはスキップ: {e}")

# 結果ファイルのダウンロードリンク表示
from google.colab import files

print("\n📁 結果ファイル:")
print(f"- 詳細結果 (JSON): {results_file}")
print(f"- サマリー (TXT): {summary_file}")
print("\n💾 ファイルをダウンロードするには以下を実行:")
print(f"files.download('{results_file}')")
print(f"files.download('{summary_file}')")

print("\n🎉 Phase 3: GED-IG迷路ナビゲーション実験完了!")
print("📊 実験データはJSON形式とテキスト形式で保存されました。")
print("🔍 結果の詳細分析や追加実験は上記の保存されたデータを使用してください。")